In [ ]:
'''
author: Felix Hol
date: 2020 Oct 20
content: code to check presence of a mosquito in an (arbitrarly shaped) ROI
Takes tracked data from trackMosq as input and a mask indicating the ROI. 
'''

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import itertools as it
import pandas as pd
import pims
import skimage
from skimage import data, io, util
from skimage.measure import label, regionprops
from skimage.morphology import binary_dilation, erosion, dilation, opening, binary_closing, closing, white_tophat, remove_small_objects, disk, black_tophat, skeletonize, convex_hull_image
import scipy
import trackpy as tp
import pylab
import math
from joblib import Parallel, delayed
import multiprocessing
from datetime import datetime
from tqdm import tnrange, tqdm
import pickle
import glob
import cv2 as cv

In [ ]:
#### set directories where to get images and where to store output, and specifics of experiment

# dataDir = '/run/user/1001/gvfs/smb-share:server=gaia.pasteur.fr,share=%40ivi/BITES_BLOOD_BEHAVIOR/P3/200923_KPPTN/ctrl02/'
dataDir = '/mnt/DATA/biteData/P3/200930_KPPTN/denv04/'
saveDir = '/home/felix/biteData/P3/200930_processed/'
species = 'aeg'
mosAge = 21      #### mosquito age in days
mosDataName = 'aeg_denv04_0930'
frames = pims.ImageSequence(dataDir+'/*.png')

In [ ]:
trackPickleName = saveDir + mosDataName + '_tracks.pkl'
tFull = pd.read_pickle(trackPickleName)

In [ ]:
#### read image containing mask - maks can e.g. be made in ImageJ 
#### by drawing with freehand tool and using 'edit' -> 'clear outside'

masks = pims.ImageSequence('/home/felix/biteData'+'/*.png')

In [ ]:
plt.imshow(masks[1])

In [ ]:
mask = masks[1]

In [ ]:
plt.imshow(mask)

In [ ]:
cnts, hierarchy = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

In [ ]:
cnts[0]

In [ ]:
tROI = tFull.copy()
tROI['roi'] = np.nan

for index, row in tROI.iterrows():
    if cv.pointPolygonTest(cnts[0], (row.x, row.y), False) == -1.0:
        tROI.loc[index, 'roi'] = 0
    elif cv.pointPolygonTest(cnts[0], (row.x, row.y), False) == 1.0:
        tROI.loc[index, 'roi'] = 1

In [ ]:
plt.figure(figsize=(18,12))
plt.imshow(frames[5000], cmap = 'gray')
# plt.imshow(eroded)
# plt.plot(coordinatesF[:,1],coordinatesF[:,0],'r.')
plt.plot(tROI.loc[tROI['roi'] == 0].x, tROI.loc[tROI['roi'] == 0].y,'c.', alpha=0.5)
plt.plot(tROI.loc[tROI['roi'] == 1].x, tROI.loc[tROI['roi'] == 1].y,'m.', alpha=0.5)

In [ ]:
len(tROI.loc[tROI['roi'] == 1]) / np.sum(mask>0)

In [ ]:
len(tROI.loc[tROI['roi'] == 0]) / np.sum(mask<1)

In [ ]:
(len(tROI.loc[tROI['roi'] == 1]) / np.sum(mask>0)) / (len(tROI.loc[tROI['roi'] == 0]) / np.sum(mask<1))